In [ ]:
%matplotlib qt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from helpers.ssa import SSA
from helpers.blackbox_gyro import init_figure_settings,show_axes, show_axes_v2,calc_max_noise_regression

init_figure_settings()


In [2]:
#blackbox_decode btfl_001.bbl 
data = pd.read_csv('data\\btfl_001.01.csv', encoding='cp1251', decimal='.').rename(columns=lambda x: x.strip())
print(data.columns)

Index(['loopIteration', 'time (us)', 'axisP[0]', 'axisP[1]', 'axisP[2]',
       'axisI[0]', 'axisI[1]', 'axisI[2]', 'axisD[0]', 'axisD[1]', 'axisF[0]',
       'axisF[1]', 'axisF[2]', 'rcCommand[0]', 'rcCommand[1]', 'rcCommand[2]',
       'rcCommand[3]', 'setpoint[0]', 'setpoint[1]', 'setpoint[2]',
       'setpoint[3]', 'vbatLatest (V)', 'gyroADC[0]', 'gyroADC[1]',
       'gyroADC[2]', 'accSmooth[0]', 'accSmooth[1]', 'accSmooth[2]',
       'debug[0]', 'debug[1]', 'debug[2]', 'debug[3]', 'motor[0]', 'motor[1]',
       'motor[2]', 'motor[3]', 'flightModeFlags (flags)', 'stateFlags (flags)',
       'failsafePhase (flags)', 'rxSignalReceived', 'rxFlightChannelsValid'],
      dtype='object')


In [3]:
data['time (us)'] = (data['time (us)'] - data['time (us)'][0])/1000000
#print(data.iloc[0,:].values.dtype)
print(data.shape)
#example data
data[0:5]

(168606, 41)


,loopIteration,time (us),axisP[0],axisP[1],axisP[2],axisI[0],axisI[1],axisI[2],axisD[0],axisD[1],...,debug[3],motor[0],motor[1],motor[2],motor[3],flightModeFlags (flags),stateFlags (flags),failsafePhase (flags),rxSignalReceived,rxFlightChannelsValid
0,0,0.000000,0,0,0,0,0,0,0,0,...,0,138,138,138,138,0,0,IDLE,0,0
1,1,0.000570,0,0,0,0,0,0,0,0,...,0,138,138,138,138,0,SMALL_ANGLE,IDLE,0,1
2,2,0.001112,0,0,0,0,0,0,0,0,...,0,138,138,138,138,0,SMALL_ANGLE,IDLE,0,1
3,3,0.001654,0,0,0,0,0,0,0,0,...,0,138,138,138,138,0,SMALL_ANGLE,IDLE,0,1
4,4,0.002186,0,0,0,0,0,0,0,0,...,0,138,138,138,138,0,SMALL_ANGLE,IDLE,0,1


In [4]:
#time step
data['time (us)'].diff()[0:5]

0         NaN
1    0.000570
2    0.000542
3    0.000542
4    0.000532
Name: time (us), dtype: float64

In [7]:
start=0
stop=len(data)
samplerate=2048.0
NFFT=1024

In [9]:
show_axes_v2(data,start,stop,samplerate,['debug[0]','debug[1]','debug[2]'],'gyro_scaled',NFFT)
show_axes_v2(data,start,stop,samplerate,['gyroADC[0]','gyroADC[1]','gyroADC[2]'],'filtered gyro',NFFT)

show_axes_v2(data,start,stop,samplerate,['debug[2]'],'gyro_scaled',NFFT)
show_axes_v2(data,start,stop,samplerate,['gyroADC[2]'],'filtered gyro',NFFT)


In [ ]:
#draw metric against time
plt.figure()
plt.plot(data['time (us)'], data['axisD[0]'])

In [11]:
#try to find linear dependency between main RPM harmonic (from X axis) and aliased yaw noise(Z axis)
start = 45000
stop = 52000
calc_max_noise_regression(data,start,stop,samplerate,'debug[0]','debug[2]',1024)

LinregressResult(slope=-13.411585365853659, intercept=3221.591463414634, rvalue=-0.995668499347377, pvalue=2.3661446870760353e-06, stderr=0.5600718462240183)


In [12]:
#experimental: try to separate flight data and vibrations with singular-spectrum analysis
series = data['debug[2]'][start:stop].values
print(series[10:40])
L = 400
ssa = SSA(series, L)
ssa.components_to_df(10).plot()
ssa.orig_TS.plot(alpha=0.4)
plt.xlabel("$t$")
plt.ylabel(r"$\tilde{F}_i(t)$")
plt.title("ssa");

[-25   6  27   8 -31 -17  18  26 -12 -31   0  28  20 -22 -28  10  32   9
 -21 -13  20  34  -6 -31  -9  21  12 -19 -32   3]


In [13]:
#correlation
plt.figure()
plt.subplot(1,2,1)
ssa.plot_wcorr()
plt.subplot(1,2,2)
ssa.plot_wcorr(0,12)

In [ ]:
#contribution of components 
ssa.plot_comp_contrib()
#plt.tight_layout()

In [ ]:
groups = [[0,1],[2,3],[4,5],[6],[7,8],[9,10,11,12]]
ssa.plot_groups(groups)
plt.tight_layout()

In [ ]:
plt.figure()
residue_components = [2,3,4,5,7,8]
residue = ssa.reconstruct(residue_components)
residue.plot()
plt.figure()
from scipy import signal
powerSpectrum, freqencies, time, imageAxis = plt.specgram(residue, Fs=samplerate, NFFT=512)
